## Programming for Data Science 
### Project Notebook: "Where should I live?" 
#### Group Members:
- Afonso Fernandes / 20241710
- Lourenço Lima / 20241711
- Pedro Jorge / 20241819
- David Morais / 20241759
## Project Repository
GitHub Repository:  
https://github.com/afonsolince06/-Where-should-I-live-PDS-Project


### Introduction

In this part of the project, the goal is to build an interactive map of Europe that allows users to explore key information about major European cities. The task combines web scraping, data integration, and geospatial visualization to create an informative and interactive tool.

To accomplish this, we will:

-Scrape the geographical coordinates of each city directly from the Wikipedia Main Page, ensuring accuracy and consistency with the provided dataset.

-Match the scraped coordinates with the dataset entries so that each city is correctly assigned to its corresponding country, population, average salary, and cost of living.

-Use the cleaned and enriched dataset to construct an interactive map of Europe, where each city can be clicked or hovered over to display its relevant information.

By the end of this section, we will have a fully functional map that visually represents European cities and provides meaningful insights through an intuitive interface. This builds on the skills developed earlier in the project and introduces new concepts in geospatial data handling and visualization.

#### Import essential libraries and define an alias for them

In [22]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
from bs4 import BeautifulSoup       # process html
from selenium import webdriver      # automate web browser interaction
from selenium.webdriver.common.by import By #specify specify how to locate elements on a web page.
import requests                    # make requests to fetch web pages
import time


In [23]:

urll=  'https://en.wikipedia.org/wiki/Main_Page'
from selenium.webdriver.chrome.options import Options as Options_c

options = Options_c()
browser_c = webdriver.Chrome(options=Options_c())
browser_c.get(urll)



In [24]:
search_button = browser_c.find_elements(By.CLASS_NAME, "cdx-text-input__input")
search_button

[<selenium.webdriver.remote.webelement.WebElement (session="788fbcce825219e47b25d0ed140ac8cb", element="f.3CB89972841E0FB693B27E45D461300F.d.5783C477A60C3C34DF0D7BE40153EA95.e.31")>,
 <selenium.webdriver.remote.webelement.WebElement (session="788fbcce825219e47b25d0ed140ac8cb", element="f.3CB89972841E0FB693B27E45D461300F.d.5783C477A60C3C34DF0D7BE40153EA95.e.39")>]

In [25]:
search_button[0].click()
from selenium.webdriver.common.keys import Keys
search_box = search_button[0]
search_box.send_keys("Vienna")

# Optionally, press Enter to submit the search
search_box.send_keys(Keys.RETURN)


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=142.0.7444.163); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementnotinteractableexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7d34fa235
	0x7ff7d3252630
	0x7ff7d2fe14e5
	0x7ff7d3033a13
	0x7ff7d303186a
	0x7ff7d3062b0a
	0x7ff7d302cc36
	0x7ff7d308baba
	0x7ff7d302b0ed
	0x7ff7d302bf63
	0x7ff7d3525d60
	0x7ff7d351fe8a
	0x7ff7d3541005
	0x7ff7d326d71e
	0x7ff7d3274e1f
	0x7ff7d325b7c4
	0x7ff7d325b97f
	0x7ff7d32418e8
	0x7ffc90d17374
	0x7ffc91bbcc91


In [ ]:
vienna_latitude=browser_c.find_element(By.CLASS_NAME, "latitude")
print(vienna_latitude.text)


48°12′30″N


In [ ]:
data=pd.read_csv('city_data_cleaned.csv')
cities=data['City'].tolist()
countries=data['Country'].tolist()
cities



['Salzburg',
 'Vienna',
 'Antwerp',
 'Bruges',
 'Brussels',
 'Gent',
 'Dobrich',
 'Sofia',
 'Split',
 'Zagreb',
 'Lefkosia',
 'Lemesos',
 'Ostrava',
 'Prague',
 'Copenhagen',
 'Odense',
 'Tallinn',
 'Helsinki',
 'Tampere',
 'Lyon',
 'Paris',
 'Toulouse',
 'Berlin',
 'Cologne',
 'Dresden',
 'Dusseldorf',
 'Frankfurt am Main',
 'Hamburg',
 'Hanover',
 'Leipzig',
 'Munich',
 'Stuttgart',
 'Athens',
 'Thessaloniki',
 'Budapest',
 'Debrecen',
 'Miskolc',
 'Cork',
 'Dublin',
 'Florence',
 'Milan',
 'Naples',
 'Rome',
 'Turin',
 'Venice',
 'Riga',
 'Luxembourg',
 'Malta',
 'Amsterdam',
 'Eindhoven',
 'Rotterdam',
 'The Hague',
 'Utrecht',
 'Bergen',
 'Oslo',
 'Stavanger',
 'Cracow',
 'Lodz',
 'Warsaw',
 'Braga',
 'Coimbra',
 'Lisbon',
 'Porto',
 'Giroc',
 'Bratislava',
 'Ljubljana',
 'Barcelona',
 'Madrid',
 'Malaga',
 'Seville',
 'Valencia',
 'Gothenburg',
 'Malmo',
 'Stockholm',
 'Basel',
 'Geneva',
 'Zurich',
 'Adana',
 'Ankara',
 'Edinburgh',
 'Glasgow',
 'Leeds',
 'Liverpool',
 'London']

In [ ]:
cities_1=['Salzburg',
 'Vienna',
 'Antwerp',
 'Bruges']
countries_1=['Austria',
 'Austria',
 'Belgium',
 'Belgium']

In [ ]:
def get_city_coordinates(cities):
    options = webdriver.ChromeOptions()
    browser = webdriver.Chrome(options=options)
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC


    results = {}

    for city in cities:
        browser.get("https://en.wikipedia.org/wiki/Main_Page")
        try:
            # -----------------------------
            # 1. Find search bar (fresh each loop)
            # -----------------------------
            search_bar = WebDriverWait(browser, 15).until(
                EC.element_to_be_clickable((By.ID, "searchInput"))
            )
            search_bar.clear()
            search_bar.send_keys(city)
            search_bar.send_keys(Keys.RETURN)

            # Wait for page to load (city page)
            time.sleep(10)
            WebDriverWait(browser, 15).until(
                EC.presence_of_element_located((By.ID, "firstHeading"))
            )
        

            # -----------------------------
            # 2. Extract coordinates
            # -----------------------------
            WebDriverWait(browser, 15).until(
                EC.presence_of_element_located((By.CLASS_NAME, "latitude"))
            )
            WebDriverWait(browser, 15).until(
                EC.presence_of_element_located((By.CLASS_NAME, "longitude"))
            )
            latitude_element = browser.find_element(By.CLASS_NAME, "latitude")
            longitude_element = browser.find_element(By.CLASS_NAME, "longitude")

            latitude = latitude_element.text
            longitude = longitude_element.text

            results[city] = (latitude, longitude)

        except Exception as e:
            print(f"Coordinates not found for {city}. Error: {e}")
            results[city] = None

        time.sleep(5)
        # go back to the main page for the next loop
        browser.get("https://en.wikipedia.org/wiki/Main_Page")

    browser.quit()
    return results


In [ ]:


def get_city_coordinates_1(cities):
    options = webdriver.ChromeOptions()
    # Add an argument to run Chrome headless (without a GUI) for stability
    # options.add_argument("--headless") 
    browser = webdriver.Chrome(options=options)

    results = {}

    for city in cities:
        print(f"Searching for {city}...")
        try:
            # Go to Wikipedia main page
            browser.get("https://en.wikipedia.org/wiki/Main_Page")

            # Search for the city
            search_bar = WebDriverWait(browser, 15).until(
                EC.element_to_be_clickable((By.ID, "searchInput"))
            )
            search_bar.clear()
            search_bar.send_keys(city)
            search_bar.send_keys(Keys.RETURN)

            # --- MODIFIED LOGIC START ---

            # Check if the current page is a search results page.
            # We look for the main heading 'Search results' or 'The page X was not found'.
            # If the search led directly to an article, this check will likely time out, 
            # and we move to the coordinate extraction step.

            try:
                WebDriverWait(browser, 5).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "mw-search-results"))
                )
                
                # If the search results container is found, it means we are on the results page.
                print(f"Search results page found for {city}. Clicking first link.")
                
                # Find the first result link within the results list.
                first_result = WebDriverWait(browser, 15).until(
                    # This XPath/CSS targets the first anchor (<a>) in the first list item (<li>) 
                    # of the search results (<ul>)
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "ul.mw-search-results > li:first-child a"))
                )
                first_result.click()
                time.sleep(3) # Wait for city page to load after clicking the link
                
            except TimeoutException:
                # This means the search led directly to a page (not the search results page). 
                # We assume the correct article page is loaded and continue to extraction.
                print(f"Direct article match for {city}.")
                pass # Continue to the coordinate extraction below

            # --- MODIFIED LOGIC END ---

            # Extract coordinates
            latitude_element = WebDriverWait(browser, 15).until(
                EC.presence_of_element_located((By.CLASS_NAME, "latitude"))
            )
            # Use `find_element` as `presence_of_element_located` has confirmed the page structure
            longitude_element = browser.find_element(By.CLASS_NAME, "longitude")

            latitude = latitude_element.text
            longitude = longitude_element.text

            results[city] = (latitude, longitude)
            print(f"Successfully retrieved coordinates for {city}: ({latitude}, {longitude})")

        except TimeoutException:
            print(f"Coordinates not found for {city}: Timed out while waiting for elements.")
            results[city] = None
        except NoSuchElementException:
            print(f"Coordinates not found for {city}: Element not found on page.")
            results[city] = None
        except Exception as e:
            print(f"Coordinates not found for {city}. General error: {e}")
            results[city] = None

        time.sleep(2) # Pause before next city

    browser.quit()
    return results

# Example Usage:
# city_list = ["London", "Paris", "Berlin"]
# coordinates = get_city_coordinates(city_list)
# print("\nFinal Coordinates:")
# print(coordinates)

In [ ]:


def get_city_coordinates_2(cities):
    options = webdriver.ChromeOptions()
    browser = webdriver.Chrome(options=options)
    results = {}

    for city in cities:
        try:
            browser.get("https://en.wikipedia.org/wiki/Main_Page")
            search_bar = WebDriverWait(browser, 15).until(
                EC.element_to_be_clickable((By.ID, "searchInput"))
            )
            search_bar.clear()
            search_bar.send_keys(city)
            search_bar.send_keys(Keys.RETURN)
            time.sleep(5)

            # Function to find coordinates on current page
            def find_coordinates():
                try:
                    lat = browser.find_element(By.CLASS_NAME, "latitude").text
                    lon = browser.find_element(By.CLASS_NAME, "longitude").text
                    return (lat, lon)
                except:
                    return None

            coords = find_coordinates()
            if coords:
                results[city] = coords
                continue

            # If search results page exists, click first result
            search_results = browser.find_elements(By.CSS_SELECTOR, "ul.mw-search-results li a")
            if search_results:
                search_results[0].click()
                time.sleep(2)
                coords = find_coordinates()
                if coords:
                    results[city] = coords
                    continue

            # If disambiguation, click first link and check
            links = browser.find_elements(By.CSS_SELECTOR, "div.mw-parser-output ul li a")
            for link in links:
                link.click()
                time.sleep(2)
                coords = find_coordinates()
                if coords:
                    results[city] = coords
                    break
                browser.back()
                time.sleep(2)
            if city not in results:
                results[city] = None

        except Exception as e:
            print(f"Coordinates not found for {city}. Error: {e}")
            results[city] = None

    time.sleep(5)
    browser.quit()
    time.sleep(5)
    return results



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def get_cities_coordinates(cities):
    """
    Scrapes latitude and longitude from Wikipedia for a list of cities.
    
    Args:
        cities (list): List of city names (strings)
    
    Returns:
        dict: {city_name: (latitude, longitude) or None}
    """
    url = 'https://en.wikipedia.org/wiki/Main_Page'
    
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")  # Uncomment to run headless
    browser = webdriver.Chrome(options=options)
    
    results = {}
    
    for city in cities:
        try:
            browser.get(url)
            time.sleep(2)  # wait for page to load
            
            # Correct search input
            search_input = browser.find_element(By.ID, "searchInput")
            search_input.clear()
            search_input.send_keys(city)
            search_input.send_keys(Keys.RETURN)
            
            time.sleep(3)  # wait for page to load
            
            # Extract coordinates
            latitude = browser.find_element(By.CLASS_NAME, "latitude").text
            longitude = browser.find_element(By.CLASS_NAME, "longitude").text
            results[city] = (latitude, longitude)
            print(f"{city}: ({latitude}, {longitude})")
            
        except Exception as e:
            print(f"Coordinates not found for {city}. Error: {e}")
            results[city] = None
        
        time.sleep(2)  # pause before next city
    
    browser.quit()
    return results




In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_cities_coordinates_1(cities):
    url = 'https://en.wikipedia.org/wiki/Main_Page'
    options = webdriver.ChromeOptions()
    browser = webdriver.Chrome(options=options)
    results = {}

    for city in cities:
        try:
            browser.get(url)
            time.sleep(3)

            # Search input
            search_input = browser.find_element(By.ID, "searchInput")
            search_input.clear()
            search_input.send_keys(city)
            search_input.send_keys(Keys.RETURN)
            time.sleep(3)

            # Case 1: Search results page
            try:
                first_result = browser.find_element(By.CSS_SELECTOR, "ul.mw-search-results li a")
                first_result.click()
                time.sleep(3)
            except:
                pass  # no search results list

            # Case 2: Disambiguation page
            try:
                disambig_link = browser.find_element(By.CSS_SELECTOR, "table.disambig a")
                disambig_link.click()
                time.sleep(3)
            except:
                pass  # not a disambiguation page

            # Extract coordinates
            latitude = browser.find_element(By.CLASS_NAME, "latitude").text
            longitude = browser.find_element(By.CLASS_NAME, "longitude").text
            results[city] = (latitude, longitude)
            print(f"{city}: ({latitude}, {longitude})")

        except Exception as e:
            print(f"Coordinates not found for {city}. Error: {e}")
            results[city] = None

        time.sleep(2)

    browser.quit()
    return results


In [ ]:
coordinates_city=get_cities_coordinates_1(cities)
coordinates_city

Salzburg: (47°48′00″N, 13°02′42″E)
Coordinates not found for Vienna. Error: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=142.0.7444.175); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#staleelementreferenceexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7240ba235
	0x7ff723e12630
	0x7ff723ba16dd
	0x7ff723ba94c8
	0x7ff723bac52f
	0x7ff723bac5ef
	0x7ff723bfb23a
	0x7ff723bf0d0b
	0x7ff723c22b0a
	0x7ff723becc36
	0x7ff723c4baba
	0x7ff723beb0ed
	0x7ff723bebf63
	0x7ff7240e5d60
	0x7ff7240dfe8a
	0x7ff724101005
	0x7ff723e2d71e
	0x7ff723e34e1f
	0x7ff723e1b7c4
	0x7ff723e1b97f
	0x7ff723e018e8
	0x7ffa7c85e8d7
	0x7ffa7e04c53c

Coordinates not found for Antwerp. Error: Message: element not interactable
  (Session info: chrome=142.0.7444.175); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/trouble

{'Salzburg': ('47°48′00″N', '13°02′42″E'),
 'Vienna': None,
 'Antwerp': None,
 'Bruges': None,
 'Brussels': None,
 'Gent': None,
 'Dobrich': None,
 'Sofia': None,
 'Split': None,
 'Zagreb': None,
 'Lefkosia': None,
 'Lemesos': None,
 'Ostrava': None,
 'Prague': None,
 'Copenhagen': None,
 'Odense': ('55°23′45″N', '10°23′19″E'),
 'Tallinn': ('59°26′14″N', '24°44′43″E'),
 'Helsinki': ('60°10′15″N', '24°56′15″E'),
 'Tampere': ('61°29′53″N', '23°45′36″E'),
 'Lyon': ('45°46′03″N', '4°50′06″E'),
 'Paris': ('48°51′24″N', '2°21′8″E'),
 'Toulouse': ('43°36′16″N', '1°26′38″E'),
 'Berlin': ('52°31′12″N', '13°24′18″E'),
 'Cologne': ('50°56′11″N', '6°57′10″E'),
 'Dresden': None,
 'Dusseldorf': None,
 'Frankfurt am Main': None,
 'Hamburg': None,
 'Hanover': None,
 'Leipzig': None,
 'Munich': None,
 'Stuttgart': None,
 'Athens': None,
 'Thessaloniki': None,
 'Budapest': None,
 'Debrecen': None,
 'Miskolc': None,
 'Cork': None,
 'Dublin': None,
 'Florence': None,
 'Milan': None,
 'Naples': None,
 'Rom